# 1.2 Player Salary Data Collection
---

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.spotrac.com/mlb/rankings/2021/salary/batters/'

In [3]:
data = {
    'ajax': 'true',
    'mobile': 'false'
}

res = requests.post(url, data=data)

print(res.status_code)

#Copied from https://stackoverflow.com/questions/63376961/beautifulsoup-only-returing-100-elements

200


In [4]:
soup = BeautifulSoup(res.content, 'lxml')

In [5]:
table = soup.find('table')

In [6]:
batters = []

for row in table.find('tbody').find_all('tr'):
    batter = {}
    batter['player'] = row.find('h3').text.strip()
    batter['salary'] = row.find('span', {'class':'info'}).text.strip()
    
    batters.append(batter)
    
salary = pd.DataFrame(batters)

In [7]:
df = pd.read_csv('../mlb_players.csv').drop('Unnamed: 0', axis = 1)
df.head()

,MLBID,FIRSTNAME,LASTNAME,ACTIVE,Player,Team,Pos,Age,G,AB,...,CS,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS
0,547989,Jose,Abreu,Y,Jose Abreu,CWS,1B,34,152,566,...,0,61,143,0,10,22,0.261,0.351,0.481,0.832
1,660670,Ronald,Acuna,Y,Ronald Acuna,ATL,OF,23,82,297,...,6,49,85,0,5,9,0.283,0.394,0.596,0.990
2,642715,Willy,Adames,Y,Willy Adames,TB,SS,26,41,132,...,2,10,51,0,0,0,0.197,0.254,0.371,0.625
3,666176,Jo,Adell,Y,Jo Adell,LAA,OF,22,35,130,...,1,8,32,1,0,1,0.246,0.295,0.408,0.703
4,501303,Ehire,Adrianza,Y,Ehire Adrianza,ATL,SS,32,109,182,...,0,21,42,1,3,2,0.247,0.327,0.401,0.728


In [8]:
df = df.merge(salary, how = 'inner', left_on = 'Player', right_on = 'player')
df.head()

,MLBID,FIRSTNAME,LASTNAME,ACTIVE,Player,Team,Pos,Age,G,AB,...,SO,SH,SF,HBP,AVG,OBP,SLG,OPS,player,salary
0,547989,Jose,Abreu,Y,Jose Abreu,CWS,1B,34,152,566,...,143,0,10,22,0.261,0.351,0.481,0.832,Jose Abreu,"$17,666,666"
1,642715,Willy,Adames,Y,Willy Adames,TB,SS,26,41,132,...,51,0,0,0,0.197,0.254,0.371,0.625,Willy Adames,"$590,000"
2,501303,Ehire,Adrianza,Y,Ehire Adrianza,ATL,SS,32,109,182,...,42,1,3,2,0.247,0.327,0.401,0.728,Ehire Adrianza,"$1,500,000"
3,542583,Jesus,Aguilar,Y,Jesus Aguilar,MIA,1B,31,131,449,...,93,0,7,3,0.261,0.329,0.459,0.788,Jesus Aguilar,"$4,500,000"
4,605113,Nick,Ahmed,Y,Nick Ahmed,ARI,SS,31,129,434,...,104,2,1,2,0.221,0.280,0.339,0.619,Nick Ahmed,"$8,125,000"


In [9]:
df.drop_duplicates(subset=['MLBID'], keep='first', inplace = True)

# Make sure no duplicates
df[df.duplicated(['MLBID'])]

,MLBID,FIRSTNAME,LASTNAME,ACTIVE,Player,Team,Pos,Age,G,AB,...,SO,SH,SF,HBP,AVG,OBP,SLG,OPS,player,salary


In [10]:
df.drop(columns = 'player', inplace = True)
df.head(3)

,MLBID,FIRSTNAME,LASTNAME,ACTIVE,Player,Team,Pos,Age,G,AB,...,BB,SO,SH,SF,HBP,AVG,OBP,SLG,OPS,salary
0,547989,Jose,Abreu,Y,Jose Abreu,CWS,1B,34,152,566,...,61,143,0,10,22,0.261,0.351,0.481,0.832,"$17,666,666"
1,642715,Willy,Adames,Y,Willy Adames,TB,SS,26,41,132,...,10,51,0,0,0,0.197,0.254,0.371,0.625,"$590,000"
2,501303,Ehire,Adrianza,Y,Ehire Adrianza,ATL,SS,32,109,182,...,21,42,1,3,2,0.247,0.327,0.401,0.728,"$1,500,000"


In [11]:
df.to_csv('../data/mlb_players.csv')